In [36]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import glob
import json

import pandas as pd

In [38]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/bybit/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/bybit','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = True if value == 'True' else False

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())


        big_data = big_data.append(d, ignore_index=True)

# files = glob.iglob('backtests/bybit_production/' + '**/backtest_result.txt', recursive=True)
# i = 0
# for filename in files:
#     #print(f'parsing : {filename}')
#     json_file = filename.replace('backtest_result.txt','live_config.json')
#     with open(json_file) as f:
#         live_config = json.load(f)
#         config_type = live_config['config_name']
#
#     with open(filename, 'r') as file:
#         data = file.read().split('\n')
#         d = {'file': filename.replace('backtests/bybit_production','')}
#         current_state = 'long'
#         for l in data:
#             l = l.split('|')
#             if len(l) == 4:
#                 key = l[1].strip()
#                 value = l[2].strip()
#
#                 if key == 'Passivbot Version':
#                     d['v'] = value
#                 if key == 'Symbol':
#                     d['s'] = value
#
#                 if key == 'Long':
#                     current_state = 'long'
#                     d['long'] = bool(value) or False
#                 if key == 'Short':
#                     current_state = 'short'
#                     d['short'] = bool(value) or False
#
#                 if key == 'Total gain' and current_state == 'long':
#                     d['tgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'long':
#                     d['adgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'long':
#                     d['bnkrl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'long':
#                     d['ebrl'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'long':
#                     d['maxsl'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'long':
#                     d['pamaxl'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'long':
#                     d['pastdl'] = float(value.strip())
#
#
#                 if key == 'Total gain' and current_state == 'short':
#                     d['tgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'short':
#                     d['adgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'short':
#                     d['bnkrs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'short':
#                     d['ebrs'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'short':
#                     d['maxss'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'short':
#                     d['pamaxs'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'short':
#                     d['pastds'] = float(value.strip())
#
#         d['prod'] = True
#         big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_10468/573714298.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_10468/573714298.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_10468/573714298.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_10468/573714298.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=Tr

,file,v,prod,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss
0,\ADAUSDT\plots\2022-05-07T105930\backtest_resu...,5.5,False,ADAUSDT,True,-3.789,-0.00998,0.126457,0.753290,100.0,0.8859,63.053333,True,29.160,0.08990,0.081063,0.481465,100.0,0.7875,27.359708
1,\ADAUSDT\plots\2022-05-07T152824\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,0.9222,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500
2,\ADAUSDT\plots\2022-05-07T183907\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,0.9222,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500
3,\ADAUSDT\plots\2022-05-07T191008\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,0.9222,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500
4,\ADAUSDT\plots\2022-05-07T213637\backtest_resu...,5.5,False,ADAUSDT,True,-3.789,-0.00998,0.126457,0.753290,100.0,0.8859,63.053333,True,29.160,0.08990,0.081063,0.481465,100.0,0.7875,27.359708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,\XRPUSDT\plots\2022-05-07T183703\backtest_resu...,5.5,False,XRPUSDT,True,7.317,0.01380,0.167794,0.857247,100.0,0.9413,73.737500,True,5.205,0.01710,0.103187,0.586979,100.0,0.9478,34.750000
67,\XRPUSDT\plots\2022-05-07T190631\backtest_resu...,5.5,False,XRPUSDT,True,3.379,0.00536,0.102911,0.596623,100.0,0.9546,85.002917,True,3.343,0.00977,0.111373,0.582678,100.0,0.9479,27.393750
68,\XRPUSDT\plots\2022-05-07T190833\backtest_resu...,5.5,False,XRPUSDT,True,7.317,0.01380,0.167794,0.857247,100.0,0.9413,73.737500,True,5.205,0.01710,0.103187,0.586979,100.0,0.9478,34.750000
69,\XRPUSDT\plots\2022-05-07T213341\backtest_resu...,5.5,False,XRPUSDT,True,10.340,0.03170,0.058584,0.377257,100.0,0.9418,29.554875,True,-3.943,-0.00655,0.113785,0.477048,100.0,0.7767,61.601250


Now filter the dataframe

In [39]:
# allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
#                    "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
#                    "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
#                    "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
#                    "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
#                    "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
#                    "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
#                    "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
#                    "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
#                    "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
#                    "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
#                    "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
#                    "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
#                    "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
#                    "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
#                    "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
#                    "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
#                    "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
#                    "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
#                    "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = [
    #"OMG","DUSK","GTC"
    # "LRC",
    # "--------------------------",
    # "SNX", "RSR",
    # "--------------------------",
    # "CELO", "ROSE", "OCEAN","SXP","DENT"
    # "SFP","ENS","DODO","AKRO","REEF","NEO",
    # "----------------------------------------",
    # "1000XEC","AKRO","ALPHA","ANKR","BAKE","BAND","BEL","BTS","C98","COMP",
    "-----------------------------------",
    # "DGB","DOGO","FLM","FLOW","HBAR",
    # "-----------------------------------",
    # "HOT","ICX","IOST","IOTA","KAVA","KLAY","LINA","LIT","MKR","MTL","NEO","NKN","OGN","ONT"
    #"QTUM","REEF","RLC","RVN","SC","SFP","STMX","TOMO","TRB","UNFI","WAVES", "ZIL", "ZRX"
    #"COTI"
    #"RUNE","UNI"
    #"BAT", "BAL", "SRM", "RUNE",
    #"BAT", "BAL", "SRM", "RUNE" , 'STORJ','UNI','FTM','FIL','LRC','CVC',"CTK","SKL","SAND","XEM"
   "XRP", "ADA", "SOL", "TRX", "AXS", "LUNA", "SAND", "AVAX", "FTM", "ALGO", "NEAR", "GALA", "ENS", "WAVES", "KNC", "APE", "GMT", "DAR",
]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,prod,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,...,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,adgt_%
0,\ADAUSDT\plots\2022-05-07T105930\backtest_resu...,5.5,False,ADAUSDT,True,-3.789,-0.00998,0.126457,0.753290,100.0,...,63.053333,True,29.160,0.08990,0.081063,0.481465,100.0,0.7875,27.359708,0.07992
1,\ADAUSDT\plots\2022-05-07T152824\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,...,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500,-0.10770
2,\ADAUSDT\plots\2022-05-07T183907\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,...,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500,-0.10770
3,\ADAUSDT\plots\2022-05-07T191008\backtest_resu...,5.5,False,ADAUSDT,True,-25.890,-0.13600,0.131576,0.747571,100.0,...,56.120833,True,9.241,0.02830,0.046295,0.261115,100.0,0.9611,6.562500,-0.10770
4,\ADAUSDT\plots\2022-05-07T213637\backtest_resu...,5.5,False,ADAUSDT,True,-3.789,-0.00998,0.126457,0.753290,100.0,...,63.053333,True,29.160,0.08990,0.081063,0.481465,100.0,0.7875,27.359708,0.07992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,\XRPUSDT\plots\2022-05-07T183703\backtest_resu...,5.5,False,XRPUSDT,True,7.317,0.01380,0.167794,0.857247,100.0,...,73.737500,True,5.205,0.01710,0.103187,0.586979,100.0,0.9478,34.750000,0.03090
67,\XRPUSDT\plots\2022-05-07T190631\backtest_resu...,5.5,False,XRPUSDT,True,3.379,0.00536,0.102911,0.596623,100.0,...,85.002917,True,3.343,0.00977,0.111373,0.582678,100.0,0.9479,27.393750,0.01513
68,\XRPUSDT\plots\2022-05-07T190833\backtest_resu...,5.5,False,XRPUSDT,True,7.317,0.01380,0.167794,0.857247,100.0,...,73.737500,True,5.205,0.01710,0.103187,0.586979,100.0,0.9478,34.750000,0.03090
69,\XRPUSDT\plots\2022-05-07T213341\backtest_resu...,5.5,False,XRPUSDT,True,10.340,0.03170,0.058584,0.377257,100.0,...,29.554875,True,-3.943,-0.00655,0.113785,0.477048,100.0,0.7767,61.601250,0.02515


In [40]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,\ADAUSDT\plots\2022-05-07T105930\backtest_resu...,False,ADAUSDT,0.07992,-0.00998,0.08990,100.000,100.000,0.8859,0.78750,63.053333,0.753290,0.126457,27.359708,0.481465,0.081063
1,\ADAUSDT\plots\2022-05-07T152824\backtest_resu...,False,ADAUSDT,-0.10770,-0.13600,0.02830,100.000,100.000,0.9222,0.96110,56.120833,0.747571,0.131576,6.562500,0.261115,0.046295
5,\ALGOUSDT\plots\2022-05-07T193148\backtest_res...,False,ALGOUSDT,-0.31652,-0.32300,0.00648,100.000,100.000,0.9658,0.97620,23.634042,0.577051,0.075273,21.278458,0.467667,0.070771
6,\ALGOUSDT\plots\2022-05-07T214805\backtest_res...,False,ALGOUSDT,0.10060,0.06010,0.04050,100.000,100.000,0.9316,0.76890,16.115292,0.514621,0.035039,47.290417,0.534201,0.075699
7,\APEUSDT\plots\2022-05-07T210059\backtest_resu...,False,APEUSDT,0.02880,-0.01910,0.04790,100.000,100.000,0.9453,0.89430,11.193042,0.615224,0.132037,39.729167,0.600538,0.146091
8,\APEUSDT\plots\2022-05-07T215554\backtest_resu...,False,APEUSDT,-0.33400,0.11400,-0.44800,100.000,100.000,0.9158,0.81710,10.126375,0.511342,0.106202,11.463875,0.454196,0.091128
9,\AVAXUSDT\plots\2022-05-07T173943\backtest_res...,False,AVAXUSDT,0.34200,0.17700,0.16500,100.000,43.960,0.8251,0.54090,34.609708,0.782233,0.139497,36.545125,0.823917,0.162490
12,\AVAXUSDT\plots\2022-05-07T214502\backtest_res...,False,AVAXUSDT,0.40300,0.21900,0.18400,100.000,42.080,0.8428,0.53070,34.614583,0.781051,0.148449,29.895125,0.713345,0.138027
13,\AXSUSDT\plots\2022-05-07T160614\backtest_resu...,False,AXSUSDT,0.25500,0.10400,0.15100,100.000,39.750,0.8538,0.56380,8.571542,0.578457,0.095039,7.022917,0.686182,0.101202
16,\AXSUSDT\plots\2022-05-07T214040\backtest_resu...,False,AXSUSDT,0.23740,0.08540,0.15200,100.000,11.540,0.8360,0.27910,15.695833,0.787773,0.151305,3.970137,0.638419,0.095033


In [42]:
dl_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    (dx['ebrl'] > 0.7) &
     (dx['adgl_%'] > 0.04) &
    #(dx['pastdl'] < 0.1) &
    (dx['maxsl'] < 20)
    ]
dl_proposed = dl_proposed[['file','s','adgl_%', 'bnkrl_%', 'ebrl',  'maxsl', 'pamaxl','pastdl' ]]
dl_proposed

,file,s,adgl_%,bnkrl_%,ebrl,maxsl,pamaxl,pastdl
6,\ALGOUSDT\plots\2022-05-07T214805\backtest_res...,ALGOUSDT,0.0601,100.0,0.9316,16.115292,0.514621,0.035039
8,\APEUSDT\plots\2022-05-07T215554\backtest_resu...,APEUSDT,0.1140,100.0,0.9158,10.126375,0.511342,0.106202
13,\AXSUSDT\plots\2022-05-07T160614\backtest_resu...,AXSUSDT,0.1040,100.0,0.8538,8.571542,0.578457,0.095039
16,\AXSUSDT\plots\2022-05-07T214040\backtest_resu...,AXSUSDT,0.0854,100.0,0.8360,15.695833,0.787773,0.151305
17,\DARUSDT\plots\2022-05-07T214905\backtest_resu...,DARUSDT,0.2190,100.0,0.9903,2.056946,0.150389,0.033358
18,\DARUSDT\plots\2022-05-07T215643\backtest_resu...,DARUSDT,0.3580,100.0,0.9740,1.818750,0.145421,0.030666
29,\FTMUSDT\plots\2022-05-07T214628\backtest_resu...,FTMUSDT,0.1270,100.0,0.8869,6.284708,0.778049,0.051267
33,\GMTUSDT\plots\2022-05-07T214754\backtest_resu...,GMTUSDT,0.1790,100.0,0.9395,7.352083,0.407919,0.046019
34,\GMTUSDT\plots\2022-05-07T215620\backtest_resu...,GMTUSDT,0.3800,100.0,0.9382,6.593750,0.460715,0.056203
36,\KNCUSDT\plots\2022-05-07T215456\backtest_resu...,KNCUSDT,0.0801,100.0,0.9144,15.750000,0.663173,0.069606


In [43]:
ds_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrs_%'] == 100) &
    (dx['ebrs'] > 0.7) &
     (dx['adgs_%'] > 0.04) &
    #(dx['pastdl'] < 0.1) &
    (dx['maxss'] < 20)
    ]
ds_proposed = ds_proposed[['file','s','adgs_%', 'bnkrs_%', 'ebrs',  'maxss', 'pamaxs','pastds' ]]
ds_proposed

,file,s,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds
17,\DARUSDT\plots\2022-05-07T214905\backtest_resu...,DARUSDT,0.0890,100.0,0.9930,2.211113,0.214257,0.055038
18,\DARUSDT\plots\2022-05-07T215643\backtest_resu...,DARUSDT,0.8710,100.0,0.9262,2.147221,0.263774,0.077600
19,\ENSUSDT\plots\2022-05-07T152121\backtest_resu...,ENSUSDT,0.1300,100.0,0.9402,4.196542,0.305193,0.055238
23,\ENSUSDT\plots\2022-05-07T215214\backtest_resu...,ENSUSDT,0.2470,100.0,0.8714,17.924292,0.344357,0.065024
36,\KNCUSDT\plots\2022-05-07T215456\backtest_resu...,KNCUSDT,0.0519,100.0,0.9350,14.991667,0.238605,0.038745
